In [1]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [2]:
! kaggle datasets download grassknoted/asl-alphabet

Dataset URL: https://www.kaggle.com/datasets/grassknoted/asl-alphabet
License(s): GPL-2.0
100% 1.02G/1.03G [00:48<00:00, 24.9MB/s]
100% 1.03G/1.03G [00:48<00:00, 22.5MB/s]


In [3]:
import zipfile
zipref = zipfile.ZipFile('asl-alphabet.zip','r')
zipref.extractall('/content')
zipref.close()

In [4]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout
from tensorflow.keras.regularizers import l2
from keras.applications.vgg16 import VGG16

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the ImageDataGenerator with basic transformations
datagen = ImageDataGenerator(
#     rescale=1.0 / 255,   # Normalize images to [0, 1] range
    validation_split=0.25,  # Split for training and validation
    rescale=1.0 / 255.0,          # Normalize images

)

# Load training data from the parent folder
train_data = datagen.flow_from_directory(
    '/content/asl_alphabet_train/asl_alphabet_train',  # Path to your main directory with subfolders
    target_size=(200, 200),      # Resize all images to 64x64
    batch_size=32,
    class_mode='categorical',  # Categorical labels for classes a, b, c, ...
    subset='training' # Specify as training data
)

# Load validation data
validation_data = datagen.flow_from_directory(
    '/content/asl_alphabet_train/asl_alphabet_train',
    target_size=(200,200),
    batch_size=32,
    class_mode='categorical',
    subset='validation'        # Specify as validation data
)


Found 65250 images belonging to 29 classes.
Found 21750 images belonging to 29 classes.


In [6]:
from tensorflow.keras.regularizers import l2

model = Sequential([
    Conv2D(32, (3, 3) ,activation='relu',kernel_regularizer=l2(0.001),input_shape=(200, 200, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3)  ,activation='relu',kernel_regularizer=l2(0.001)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3)  ,activation='relu',kernel_regularizer=l2(0.001)),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, kernel_regularizer=l2(0.001),activation='relu'),
    Dropout(0.5),
    Dense(29, activation='softmax')
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 198, 198, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 99, 99, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 97, 97, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 48, 48, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 46, 46, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 23, 23, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 33856)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       4,333,696 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 29)                  │           3,741 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,393,757 (16.76 MB)

 Trainable params: 4,393,757 (16.76 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# early_stopping = tf.keras.callbacks.EarlyStopping(
#     monitor='val_loss',  # Monitor validation loss
#     patience=3,          # Number of epochs with no improvement after which training will stop
#     restore_best_weights=True  # Restore the weights of the best epoch
# )

# # Train the model with early stopping
# history = model.fit(
#     train_data,
#     validation_data=validation_data,
#     epochs=20,             # Set a maximum number of epochs; training may stop earlier
#     callbacks=[early_stopping]  # Include early stopping in the callbacks
# )


In [9]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=3,          # Number of epochs with no improvement after which training will stop
    restore_best_weights=True  # Restore the weights of the best epoch
)

# Train the model with early stopping
history = model.fit(
    train_data,
    validation_data=validation_data,
    epochs=50,             # Set a maximum number of epochs; training may stop earlier
    callbacks=[early_stopping] # Include early stopping in the callbacks
)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2040/2040 ━━━━━━━━━━━━━━━━━━━━ 133s 61ms/step - accuracy: 0.2650 - loss: 2.6876 - val_accuracy: 0.4829 - val_loss: 2.0347
Epoch 2/50
2040/2040 ━━━━━━━━━━━━━━━━━━━━ 125s 56ms/step - accuracy: 0.6461 - loss: 1.4025 - val_accuracy: 0.6143 - val_loss: 1.5764
Epoch 3/50
2040/2040 ━━━━━━━━━━━━━━━━━━━━ 129s 63ms/step - accuracy: 0.7317 - loss: 1.1709 - val_accuracy: 0.6638 - val_loss: 1.4998
Epoch 4/50
2040/2040 ━━━━━━━━━━━━━━━━━━━━ 133s 65ms/step - accuracy: 0.7805 - loss: 1.0439 - val_accuracy: 0.6525 - val_loss: 1.5386
Epoch 5/50
2040/2040 ━━━━━━━━━━━━━━━━━━━━ 111s 54ms/step - accuracy: 0.8082 - loss: 0.9608 - val_accuracy: 0.6848 - val_loss: 1.3922
Epoch 6/50
2040/2040 ━━━━━━━━━━━━━━━━━━━━ 112s 55ms/step - accuracy: 0.8271 - loss: 0.9096 - val_accuracy: 0.7097 - val_loss: 1.3435
Epoch 7/50
2040/2040 ━━━━━━━━━━━━━━━━━━━━ 157s 62ms/step - accuracy: 0.8429 - loss: 0.8698 - val_accuracy: 0.7090 - val_loss: 1.3789
Epoch 8/50
2040/2040 ━━━━━━━━━━━━━━━━━━━━ 128s 63ms/step - accuracy: 0.8558 - lo

In [21]:
def predict_from_file(file_path, model, labels):
    img = keras.preprocessing.image.load_img(file_path, target_size=(200, 200))
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create a batch
    img_array  = img_array / 255.0

    predictions = model.predict(img_array)
    predicted_class_index = tf.argmax(predictions[0]).numpy()
    predicted_label = labels[predicted_class_index]

    return predicted_label,predictions[0]

In [32]:
import os

test_folder_path = '/content/asl_alphabet_test/asl_alphabet_test'  # Replace with your test folder path

labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
          'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
          'SPACE', 'DELETE', 'NOTHING']
true_labels = []
predicted_labels = []

for file_name in os.listdir(test_folder_path):
    file_path = os.path.join(test_folder_path, file_name)
    predicted_label, _ = predict_from_file(file_path, model, labels)
    true_label = file_name[0].upper()  # Assuming true label is the first character
    true_labels.append(true_label)
    predicted_labels.append(predicted_label)

    print(f"File: {file_name}, Predicted Label: {predicted_label}")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
File: J_test.jpg, Predicted Label: J
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
File: I_test.jpg, Predicted Label: I
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
File: L_test.jpg, Predicted Label: L
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
File: K_test.jpg, Predicted Label: K
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
File: space_test.jpg, Predicted Label: NOTHING
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
File: nothing_test.jpg, Predicted Label: DELETE
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
File: C_test.jpg, Predicted Label: C
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
File: U_test.jpg, Predicted Label: U
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
File: V_test.jpg, Predicted Label: V
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
File: N_test.jpg, Predicted Label: N
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
File: W_test.jpg, Predicted Label: W
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
File: R_test.jpg, Predicted Label: R
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
File: H_test.jpg, Predicted Label: H
1/1 